In [1]:
import pandas as pd
import numpy as np
import time
import gc

In [2]:
path = '/home/kai/data/kaggle/talkingdata/data/'
dtypes = {
        'ip'            : 'uint64',
        'app'           : 'uint64',
        'device'        : 'uint64',
        'os'            : 'uint64',
        'channel'       : 'uint64',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

train = pd.read_csv(path + 'train.csv',dtype=dtypes)
print('train done!')
# test = pd.read_csv(path + 'test.csv',dtype=dtypes)
# print('test done!')
# test_supplement = pd.read_csv(path + 'test_supplement.csv',dtype=dtypes)
# print('supplement done!')


train done!


In [3]:

dtypes = {
        'ip'            : 'uint64',
        'app'           : 'uint64',
        'device'        : 'uint64',
        'os'            : 'uint64',
        'channel'       : 'uint64',
        'is_attributed' : 'uint8',
        'click_id'      : 'uint32'
        }

train_final = pd.read_csv(path + 'train_cleaned_final.csv',dtype=dtypes)
print('train done!')

train done!


In [10]:
exp = train[train.is_attributed==1].copy()

In [11]:
exp.columns

Index(['ip', 'app', 'device', 'os', 'channel', 'click_time', 'attributed_time',
       'is_attributed'],
      dtype='object')

# Process Timestamp

# add time features

In [14]:
# get timestamp

import multiprocessing

def _apply_df(args):
    df, func, kwargs = args
    return df.apply(func, **kwargs)

def apply_by_multiprocessing(df, func, **kwargs):
    workers = kwargs.pop('workers')
    pool = multiprocessing.Pool(processes=workers)
    result = pool.map(_apply_df, [(d, func, kwargs)
            for d in np.array_split(df, workers)])
    pool.close()
    return pd.concat(list(result))

def get_timestamp(x):
    return x.timestamp()





workers = 30

for df in [exp]:
    clicks = pd.to_datetime(df.click_time)
    print('get clicks')
    df['timestamp'] = apply_by_multiprocessing(clicks, get_timestamp, workers=workers)
#     df['timestamp'] = clicks.apply(lambda t: t.timestamp())
    df['timestamp'] = df['timestamp'].astype('uint32')
    print('timestamping is done')
    
    dt = clicks.dt

    
    print('================================================================')

get clicks
timestamping is done


In [16]:
exp['timediff'] = exp['attributed_timestamp'] - exp['timestamp']

In [20]:
result = exp.groupby('channel').timediff.mean()

In [ ]:
resu